<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

In de huidige code gebruiken we de Openrouteservice API die publiekelijk gehost wordt. Deze heeft een aantal restricties. Zie https://openrouteservice.org/restrictions/.
De enige restrictie waar we in deze code last van hebben is het maximaal aantal locaties dat meegenomen kan worden in de optimalisatie. Volgens de documentatie is dat 50. Maar uit mijn testen blijkt het 70 te zijn.

Verder gebruiken we de Python client (https://openrouteservice-py.readthedocs.io/en/latest/) en construeren we dus niet zelf de curl calls.

## Prepare the script with exceptions and settings

Define the custom exceptions

In [0]:
#| echo: false
#| output: asis
show_doc(RouteCreationError)

---

### RouteCreationError



*Exception raised when a route cannot be created.*

Get the settings from the settings.yaml file.

Get the data from the Azure SQL database in a pandas DataFrame or from a previously stored pickle file.

In [ ]:
df_grouped.groups.keys()

dict_keys(['---', 'Bagger Dedemsvaart', 'Baggeren Beilervaart', 'Beekmaatregelen Reest', 'Boetelerveld_nw', 'Brongeb VL-AA', 'Dijken', 'Droogte Onderzoek NL fase 2 - bodemvochtmetingen', 'Droogtemeetnet WDOD', 'Ecologisch Effect Beekherstel Middenloop Vledder Aa Fase 1', 'Holtingerveld _Ootmaanlanden en Koningschut', 'Inrichting Dwingelderveld', 'Koekoekspolder grondwatermeetnet', 'Mastenbroek-IJssel', 'Meetnet De Wijk II', 'Nieuwveense landen', 'Nijstad Hoogeveen', 'Olde Maten en Veerslootlanden', 'Oldematen Reevediep', 'Oude Diep', 'Oude Diep Mantinge Bos_Zand', 'Oude Diep_Roode Brand', 'Oude Willem', 'Overijsselskanaal Deventer-Raalte', 'Paddenpol Zwolle-Olst', 'Primair meetnet blok 1', 'Primair meetnet blok 2', 'Primair meetnet blok 3', 'Randzone Ossenzijl Steenwijk', 'Reevediep', 'Reparatie meetpunten', 'Steenwijk_Kallenkote', 'Varsenerveld', 'Vecht', 'Vechterweerd', 'Vledder en Leierhooilanden', 'Wabos-KRW', 'ZUIDWOLDE-ZUID', 'Zandwetering Olst-Zuid_waterberging', 'dijken Stadsdi

In [ ]:
# Test one project group
test_df_group = df_grouped.get_group('Boetelerveld_nw')
test_name_group = 'Boetelerveld_nw'

# Test random sample
df_random_test = peilbuizen_df.sample(n=35)

In [ ]:
test_df_group.head()

,Id,Longitude,latitude,latest_measure_date,project
30,27HG800A;412187,6.314479,52.369582,2024-03-28 09:58:57,Boetelerveld_nw
31,27HG901A;412271,6.335889,52.366841,2024-03-28 10:27:15,Boetelerveld_nw
98,28CG001A;414791,6.342413,52.363634,2024-03-28 11:46:21,Boetelerveld_nw
243,27HG70G;411935,6.339147,52.362645,2024-03-28 11:11:21,Boetelerveld_nw
244,27HG902C;412313,6.340380,52.353098,2024-03-29 07:22:01,Boetelerveld_nw


## Start calculating the routes and storing the results

For every project in the peilbuizen dataframe, the shortest route is calculated and saved to an Excel file and an URL file.

In [0]:
#| echo: false
#| output: asis
show_doc(create_group_route)

---

### create_group_route

>      create_group_route (start_address:str,
>                          group_df:pandas.core.frame.DataFrame,
>                          route_profile:str, project_name:str,
>                          output_dir:str='output', current_date:str=None)

*Create optimized route for a group and save results*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| start_address | str |  | De startlocatie en eindlocatie opgegeven als adres |
| group_df | DataFrame |  | De pandas dataframe met peilbuizen in de groep, bijv. op basis van project |
| route_profile | str |  | Het route profiel (transport methode) waarvoor de route berekend moet worden |
| project_name | str |  | Naam van de groep, bijv. Project Vecht |
| output_dir | str | output | Locatie waar de resultaten en log-file opgeslagen worden. Als niks wordt opgegeven komen de resultaten in de output folder. |
| current_date | str | None | De datum. Als niks wordt opgegeven, wordt de huidige datum gebruikt. |
| **Returns** | **None** |  |  |

In [ ]:
create_group_route(start_address=settings['calculation']['startlocation'],
                    group_df=test_df_group,
                    route_profile=settings['calculation']['distance_calculation_method'],
                    project_name=test_name_group,
                    output_dir=Path(settings['files']['path_results']) / 'test')

In [0]:
#| echo: false
#| output: asis
show_doc(process_peilbuizen_routes)

---

### process_peilbuizen_routes

>      process_peilbuizen_routes (df:pandas.core.frame.DataFrame,
>                                 start_address:str, route_profile:str,
>                                 output_dir:str='output')

*Process peilbuizen dataframe, create optimized routes for each group,
and save results to Excel and URL files

Args:
    df: DataFrame with peilbuizen data
    start_address: Starting location address
    route_profile: De route profiel (transport methode) waarvoor de route berekend moet worden
    output_dir: Directory to save output files*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DataFrame |  | Dataframe van alle peilbuis groepen waarvoor route berekend wordt |
| start_address | str |  | De startlocatie en eindlocatie opgegeven als adres |
| route_profile | str |  | De route profiel (transport methode) waarvoor de route berekend moet worden |
| output_dir | str | output | Locatie waar de resultaten en log-file opgeslagen worden. Als niks wordt opgegeven komen de resultaten in de output folder. |
| **Returns** | **None** |  |  |

In [ ]:
# Test run voor enkele project van een deel van alle peilbuizen
process_peilbuizen_routes(df=df_random_test,
                          start_address=settings['calculation']['startlocation'],
                          route_profile=settings['calculation']['distance_calculation_method'],
                          output_dir=Path(settings['files']['path_results']) / 'test')

Processing peilbuis routes per project.:   0%|          | 0/8 [00:00<?, ?it/s]2025-02-03 11:49:14:INFO:project.utils:Project Holtingerveld _Ootmaanlanden en Koningschut heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 11:49:14:INFO:project.utils:Project Mastenbroek-IJssel heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 11:49:14:INFO:project.utils:Project Olde Maten en Veerslootlanden heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 11:49:14:INFO:project.utils:Project Primair meetnet blok 1 heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 11:49:14:INFO:project.utils:Project Primair meetnet blok 2 heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 11:49:15:INFO:project.utils:Project Primair meetnet blok 3 heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 11:49:15:INFO:project.utils:Project Steenwijk_Kallenkote heeft 3 of min

Project Holtingerveld _Ootmaanlanden en Koningschut heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Mastenbroek-IJssel heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Olde Maten en Veerslootlanden heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Primair meetnet blok 1 heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Primair meetnet blok 2 heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Primair meetnet blok 3 heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Steenwijk_Kallenkote heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Vecht heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
